## __Lab 2: Naive Bayes Classifier__

__Lebohang Mosia : 2320396__ 

__Lusanda Mdhlalose : 2332720__ 

__Thapelo Duma : 2493083__

In [1624]:
!pip install nltk

In [1625]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leboh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1626]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
import os
import pandas as pd

# nltk.download('punkt')
# nltk.download('punkt_tab')


In [1627]:
input_file_path = r"C:\Users\leboh\Downloads\harry_potter(1)\HP1.txt"

# Function to clean and split text
def clean_and_split_text(file_path):
    # Read the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Remove punctuation (including speech marks)
    text = re.sub(r'[^\w\s]', '', text.lower())
    
    # Split text by whitespace
    words = text.split()
    
    return words

# Get the cleaned and split words
words = clean_and_split_text(input_file_path)
# testWords = clean_and_split_text(input_test)

print(words)

['mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense', 'mr', 'dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'grunnings', 'which', 'made', 'drills', 'he', 'was', 'a', 'big', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', 'mrs', 'dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual', 'amount', 'of', 'neck', 'which', 'came', 'in', 'very', 'useful', 'as', 'she', 'spent', 'so', 'much', 'of', 'her', 'time', 'craning', 'over', 'garden', 'fences', 'spying', 'on', 'the', 'neighbors', 'the', 'dursley', 's', 'had', 'a', 'small', 'son', 'called', 'd

# __Question 3__

In [1629]:
# Function to tokenize words
def tokenize_words(words):
    return word_tokenize(' '.join(words))


# __Question 4__

In [1631]:
# Function to extract N-grams
def extract_ngrams(tokens, n):
    n_grams = list(ngrams(tokens, n))
    return n_grams

# Process the text
words = clean_and_split_text(input_file_path)
tokens = tokenize_words(words)
bigrams = extract_ngrams(tokens, 3)

print("Tokens:", tokens)
print("Bigrams:", bigrams)

Tokens: ['mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense', 'mr', 'dursley', 'was', 'the', 'director', 'of', 'a', 'firm', 'called', 'grunnings', 'which', 'made', 'drills', 'he', 'was', 'a', 'big', 'beefy', 'man', 'with', 'hardly', 'any', 'neck', 'although', 'he', 'did', 'have', 'a', 'very', 'large', 'mustache', 'mrs', 'dursley', 'was', 'thin', 'and', 'blonde', 'and', 'had', 'nearly', 'twice', 'the', 'usual', 'amount', 'of', 'neck', 'which', 'came', 'in', 'very', 'useful', 'as', 'she', 'spent', 'so', 'much', 'of', 'her', 'time', 'craning', 'over', 'garden', 'fences', 'spying', 'on', 'the', 'neighbors', 'the', 'dursley', 's', 'had', 'a', 'small', 'son', 'cal

# Question 5

In [1633]:
def read_pages(file_name, num_pages=None):
    words = clean_and_split_text(file_name)
    num_total_pages = len(words) // words_per_page + (1 if len(words) % words_per_page > 0 else 0)
    
    if num_pages is None or num_pages > num_total_pages:
        num_pages = num_total_pages
    
    pages = []
    for i in range(num_pages):
        start_index = i * words_per_page
        end_index = start_index + words_per_page
        page_words = words[start_index:end_index]
        tokens = tokenize_words(page_words)
        bigrams = extract_ngrams(tokens, 2)
        pages.append(bigrams)
    
    return pages

def split_data(pages):
    total_pages = len(pages)
    np.random.shuffle(pages)  # Shuffle pages to ensure random splits

    train_end = int(total_pages * training_page_percentage)
    validation_end = train_end + int(total_pages * val_page_percentage)
    
    train_pages = pages[:train_end]
    validation_pages = pages[train_end:validation_end]
    test_pages = pages[validation_end:]

    return train_pages, validation_pages, test_pages

def prepare_data(books_dir, book_names, num_pages_per_book=None):
    all_train_pages = []
    all_validation_pages = []
    all_test_pages = []

    for idx, book_name in enumerate(book_names, start=1):
        file_path = os.path.join(books_dir, book_name + ".txt")
        pages = read_pages(file_path, num_pages_per_book)

        train_pages, validation_pages, test_pages = split_data(pages)

        # Append data with corresponding targets
        all_train_pages.extend([(page, idx) for page in train_pages])
        all_validation_pages.extend([(page, idx) for page in validation_pages])
        all_test_pages.extend([(page, idx) for page in test_pages])

    return all_train_pages, all_validation_pages, all_test_pages

def create_dataframes(train_pages, validation_pages, test_pages, book_names):
    def create_dataframe(pages):
        rows = []
        for page, book_idx in pages:
            for ngram in page:
                rows.append({
                    'title': book_names[book_idx - 1],
                    'ngrams': ngram,
                    'page_number': pages.index((page, book_idx)) + 1,  # Adjust index as needed
                    'label': book_idx  # Label as the book index
                })
        return pd.DataFrame(rows)

    train_df = create_dataframe(train_pages)
    validation_df = create_dataframe(validation_pages)
    test_df = create_dataframe(test_pages)

    return train_df, validation_df, test_df


books_dir = r"C:\Users\leboh\Downloads\harry_potter(1)"
book_names = ["HP1", "HP2", "HP3", "HP4", "HP5", "HP6", "HP7"]
num_books = len(book_names)
training_page_percentage = 0.7
val_page_percentage = 0.2
testing_page_percentage = 0.1
words_per_page = 220
num_pages_per_book = 6

# Prepare data
train_pages, validation_pages, test_pages = prepare_data(books_dir, book_names, num_pages_per_book)

# Create dataframes
train_df, validation_df, test_df = create_dataframes(train_pages, validation_pages, test_pages, book_names)


train_df.to_csv('train_data2.csv', index=False)
validation_df.to_csv('validation_data2.csv', index=False)
test_df.to_csv('test_data2.csv', index=False)


In [1634]:
print("Training DataFrame Sample:")
train_df


Training DataFrame Sample:


,title,ngrams,page_number,label
0,HP1,"(sister, he)",1,1
1,HP1,"(he, didnt)",1,1
2,HP1,"(didnt, blame)",1,1
3,HP1,"(blame, her)",1,1
4,HP1,"(her, if)",1,1
...,...,...,...,...
6127,HP7,"(not, discovered)",28,7
6128,HP7,"(discovered, before)",28,7
6129,HP7,"(before, he)",28,7
6130,HP7,"(he, has)",28,7


In [1635]:
train_df.tail(20)

,title,ngrams,page_number,label
6112,HP7,"(can, all)",28,7
6113,HP7,"(all, work)",28,7
6114,HP7,"(work, together)",28,7
6115,HP7,"(together, to)",28,7
6116,HP7,"(to, bring)",28,7
6117,HP7,"(bring, scrimgeour)",28,7
6118,HP7,"(scrimgeour, down)",28,7
6119,HP7,"(down, as)",28,7
6120,HP7,"(as, long)",28,7
6121,HP7,"(long, as)",28,7


In [1636]:
print("Validation DataFrame Sample:")
validation_df.head(20)

Validation DataFrame Sample:


,title,ngrams,page_number,label
0,HP1,"(to, look)",1,1
1,HP1,"(look, again)",1,1
2,HP1,"(again, there)",1,1
3,HP1,"(there, was)",1,1
4,HP1,"(was, a)",1,1
5,HP1,"(a, tabby)",1,1
6,HP1,"(tabby, cat)",1,1
7,HP1,"(cat, standing)",1,1
8,HP1,"(standing, on)",1,1
9,HP1,"(on, the)",1,1


In [1637]:
validation_df.tail(20)


,title,ngrams,page_number,label
1513,HP7,"(voldemort, indicating)",7,7
1514,HP7,"(indicating, the)",7,7
1515,HP7,"(the, seat)",7,7
1516,HP7,"(seat, on)",7,7
1517,HP7,"(on, his)",7,7
1518,HP7,"(his, immediate)",7,7
1519,HP7,"(immediate, right)",7,7
1520,HP7,"(right, yaxley)",7,7
1521,HP7,"(yaxley, beside)",7,7
1522,HP7,"(beside, dolohov)",7,7


In [1638]:
print("Test DataFrame Sample:")
test_df.head(20)

Test DataFrame Sample:


,title,ngrams,page_number,label
0,HP1,"(together, mr)",1,1
1,HP1,"(mr, dursley)",1,1
2,HP1,"(dursley, was)",1,1
3,HP1,"(was, enraged)",1,1
4,HP1,"(enraged, to)",1,1
5,HP1,"(to, see)",1,1
6,HP1,"(see, that)",1,1
7,HP1,"(that, a)",1,1
8,HP1,"(a, couple)",1,1
9,HP1,"(couple, of)",1,1


In [1639]:
test_df.tail(20)

,title,ngrams,page_number,label
1513,HP7,"(and, ornate)",7,7
1514,HP7,"(ornate, table)",7,7
1515,HP7,"(table, the)",7,7
1516,HP7,"(the, rooms)",7,7
1517,HP7,"(rooms, usual)",7,7
1518,HP7,"(usual, furniture)",7,7
1519,HP7,"(furniture, had)",7,7
1520,HP7,"(had, been)",7,7
1521,HP7,"(been, pushed)",7,7
1522,HP7,"(pushed, carelessly)",7,7


# Question 6

In [1641]:
print(train_df['label'].value_counts())


label
1    876
2    876
3    876
4    876
5    876
6    876
7    876
Name: count, dtype: int64


In [1642]:
import os

# Save the dataframes to CSV
train_df.to_csv('train_data.csv', index=False)
validation_df.to_csv('validation_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

# Get the full paths to the CSV files
train_csv_path = os.path.abspath('train_data.csv')
validation_csv_path = os.path.abspath('validation_data.csv')
test_csv_path = os.path.abspath('test_data.csv')

# Print the file paths
print(f"Train CSV Path: {train_csv_path}")
print(f"Validation CSV Path: {validation_csv_path}")
print(f"Test CSV Path: {test_csv_path}")


Train CSV Path: C:\Users\leboh\train_data.csv
Validation CSV Path: C:\Users\leboh\validation_data.csv
Test CSV Path: C:\Users\leboh\test_data.csv


In [1643]:
import os

train_file_path = r'C:\Users\leboh\train_data2.csv'
validation_file_path = r'C:\Users\leboh\validation_data2.csv'
test_file_path = r'C:\Users\leboh\test_data2.csv'

print(os.path.exists(train_file_path))  # This should print True if the file exists
print(os.path.exists(validation_file_path))
print(os.path.exists(test_file_path))


True
True
True


In [1644]:
import pandas as pd
import numpy as np
from collections import defaultdict
import re

# Read the CSV files
train_df = pd.read_csv(r'C:\Users\leboh\train_data2.csv')
validation_df = pd.read_csv(r'C:\Users\leboh\validation_data2.csv')
test_df = pd.read_csv(r'C:\Users\leboh\test_data2.csv')

# Convert ngrams from tuples stored as strings back to tuples
train_df['ngrams'] = train_df['ngrams'].apply(eval)
validation_df['ngrams'] = validation_df['ngrams'].apply(eval)
test_df['ngrams'] = test_df['ngrams'].apply(eval)

# Function to build frequency tables from DataFrame
def build_frequency_tables(df):
    word_freq = defaultdict(lambda: defaultdict(int))
    vocab = set()

    for index, row in df.iterrows():
        ngram = row['ngrams']
        label = row['label']
        for word in ngram:
            vocab.add(word)
            word_freq[label][word] += 1
    
    return word_freq, vocab

# Calculate probabilities with Laplace smoothing
def calculate_probabilities(word_freq, vocab, laplace=1):
    total_pages = sum(sum(word_freq[label].values()) for label in word_freq)
    prior_probs = {label: sum(word_freq[label].values()) / total_pages for label in word_freq}
    
    word_probs = defaultdict(dict)
    
    for label in word_freq:
        total_words_in_label = sum(word_freq[label].values())
        for word in vocab:
            word_probs[label][word] = (word_freq[label][word] + laplace) / (total_words_in_label + laplace * len(vocab))
    
    return prior_probs, word_probs

# Predict the book label
def predict_label(ngrams, prior_probs, word_probs, vocab):
    label_scores = {}
    
    for label in prior_probs:
        label_scores[label] = np.log(prior_probs[label])  # Start with the prior probability
        
        for word in ngrams:
            if word in vocab:
                label_scores[label] += np.log(word_probs[label].get(word, 1 / (len(vocab) + 1)))  # Add log probabilities with Laplace smoothing
    
    return max(label_scores, key=label_scores.get)

# Calculate accuracy for a given DataFrame
def calculate_accuracy(df, prior_probs, word_probs, vocab):
    correct = 0
    total = 0
    for index, row in df.iterrows():
        ngrams = row['ngrams']
        true_label = row['label']
        predicted_label = predict_label(ngrams, prior_probs, word_probs, vocab)
        if predicted_label == true_label:
            correct += 1
        total += 1
    return correct / total

# Train the Naive Bayes model
word_freq_train, vocab_train = build_frequency_tables(train_df)
prior_probs_train, word_probs_train = calculate_probabilities(word_freq_train, vocab_train)

# Calculate accuracies
train_accuracy = calculate_accuracy(train_df, prior_probs_train, word_probs_train, vocab_train)
validation_accuracy = calculate_accuracy(validation_df, prior_probs_train, word_probs_train, vocab_train)
test_accuracy = calculate_accuracy(test_df, prior_probs_train, word_probs_train, vocab_train)

# Print results
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Training Accuracy: 69.47%
Validation Accuracy: 25.70%
Test Accuracy: 23.48%
